In [7]:
from data_cube_analysis import getdata, rms, masks, moments
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import spectral_cube
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [12]:
def get_moms(inputfile, velo=np.array([-200,200]), rmsvelo=[[-300,-200],[200,300]], hthresh=5, lthresh=2):
    '''
    This function computes the moments (integrated properties) of spectral line data from a FITS cube.
    
    Args:
    inputfile (str): Path to the input FITS file.
    velo (np.array): Array representing velocity bounds for the calculation of moments. Default is [-200,200].
    rmsvelo (list): List of two lists, each representing a pair of velocity bounds for the calculation of root mean square (RMS). Default is [[-300,-200],[200,300]].
    hthresh (float): High threshold value for the mask. Default is 5.
    lthresh (float): Low threshold value for the mask. Default is 2.
    
    Returns:
    tuple: Tuple containing the original header-data unit (HDU), the data cube, the RMS map, and the moment maps.
    '''

    # Use the get_cube function from the getdata module to extract the data cube from the FITS file.
    cube = getdata.get_cube(inputfile)

    # Use the get_rms function from the rms module to compute the RMS map of the cube over the specified velocity range.
    rms_map = rms.get_rms(cube, rmsvelo)
    
    # Use the get_expmask function from the masks module to generate a mask for the cube using the specified thresholds and the RMS map.
    mask = masks.get_expmask(cube, rms_map, hthresh=hthresh, lthresh=lthresh, npix=2)
    
    # Apply the mask to the cube.
    cube_masked = cube.with_mask(mask)
    
    # Use the get_momentmaps function from the moments module to compute moment maps of the masked cube over the specified velocity range.
    moms = moments.get_momentmaps(cube_masked, rms_map, velo, velomoms=True)
   
    # Define the output file name by removing '.fits' from the input file name.
    outputfile = inputfile.replace('.fits', '')
    
    # Write the zeroth moment map (integrated intensity) to a FITS file.
    moms['mom0'].writeto('%s.mom0.fits' %outputfile, overwrite=True)
    
    # Write the first moment map (velocity) to a FITS file.
    moms['mom1'].writeto('%s.mom1.fits' %outputfile, overwrite=True)
    
    # Write the maximum intensity map to a FITS file.
    moms['max'].writeto('%s.max.fits' %outputfile, overwrite=True)
    
    # Write the signal-to-noise ratio map of the zeroth moment to a FITS file.
    moms['mom0s2n'].writeto('%s.mom0_s2n.fits' %outputfile, overwrite=True)
    
    # Write the second moment map to a FITS file.
    moms['sigma'].writeto('%s.sigma.fits' %outputfile, overwrite=True)
    
    # Write the RMS to a FITS file.
    rms_map.writeto('%s.rms.fits' %outputfile, overwrite=True)
    
    # Return the original HDU, the data cube, the RMS map, and the moment maps.
    return(cube, rms_map, moms)

In [13]:
inputfile = '/Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma/12m7mtp_bubble/hnco43.TP_7M_12M_weighted_mosaic.K.kms.galactic.fits'
rmsvelo = [[-44.5,-37], [123,130]]
momvelo = np.array([-5,75])

cube, rms_map, moms = get_moms(inputfile, momvelo, rmsvelo)

In [14]:
inputfile = '/Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma/12m7mtp_bubble/hnco43.TP_7M_12M_weighted_mosaic.rebin.K.kms.0.5kmschannel.galactic.fits'
rmsvelo = [[-44.5,-37], [123,130]]
momvelo = np.array([-5,75])

cube, rms_map, moms = get_moms(inputfile, momvelo, rmsvelo)